<img src="Figs/Banner.JPG" width="100%" />
<br>
<font size="7"> <b> GEOS 639 Geodetic Imaging </b> </font>

<font size="5"> <b>Lab 6: Glacier Velocity Mapping using Template Matching </b> </font>

<br>
<font size="4"> <b> Franz J Meyer; University of Alaska Fairbanks </b> <br>

</font>

<img style="padding: 0px 0px 0px 10px" src="Figs/UAFLogo_A_647.png" width="170" align="right" /> <font size="3"> This lab will let you exercise template matching techniques for the application of glacier velocity mapping. We will initially perform template matching using cross-correlation techniques on a pair of Sentinel-2 optical images covering an are of south east Alaska that includes Malaspina and Hubbard Glaciers. Subsequently, we will work with a large number of Sentinel-1 image pairs processed using the AutoRIFT algorithm covering the same area. This larger set of images will help understand recent changes in glacier velocity at these glaciers. 
    
The notebook will discuss error sources of template matching techniques near the end of the workflow.

<b>This Lab is part of the UAF course GEOS639 Geodetic Imaging. It will introduce the following data analysis concepts:</b>

- Using template matching for the application of glacier velocity mapping
- How to order template matching-based offset maps using the AutoRIFT algorithm 
- Analysis of glacier velocity changes at Maimport url_widget as url_w
notebookUrl = url_w.URLWidget()
display(notebookUrl)laspina glacier, Alaska.
</font>
<br>
</font>

<div class="alert alert-success">
<font size="4"> <b>THIS NOTEBOOK DOES NOT INCLUDE A HOMEWORK ASSIGNMENTS.</b></font> 
<br> 
<font size="3"> This Lab is allowing you to exercise template-based tracking technology and gets you exposed to some of the datasets coming out of template matching workflows. There is no homework assignment attached to this lab.</font> <br>
</font>
</div>

In [ ]:
import url_widget as url_w
notebookUrl = url_w.URLWidget()
display(notebookUrl)

In [ ]:
from IPython.display import Markdown
from IPython.display import display
import netCDF4 as nc
notebookUrl = notebookUrl.value
user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/unavco':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "unavco" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select "unavco" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "unavco" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

# 0. Importing Relevant Python Packages and Cloning Necessary GitHub Repositories

First step in any notebook is to import the required Python libraries into the Jupyter environment. In this notebooks we use the following scientific libraries:
<ol type="1">
    <li> <b><a href="http://www.numpy.org/" target="_blank">NumPy</a></b> is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays. </li>
    <li> <b><a href="https://matplotlib.org/index.html" target="_blank">Matplotlib</a></b> is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs. </li>
</ol>
<br>
In addition to that, we will also <b>import clone a <a href="https://github.com/markf6/pycorr_iceflow" target="_blank">GitHub repository</a></b> generated by Mark Fahnestock, UAF that includes glacier tracking code that is using cross-correlation analysis implemented in a rather lightweight openCV framework</b>

In [ ]:
!conda install -c conda-forge fiona --yes

In [ ]:
from osgeo import gdal
import os
import numpy as np
from math import ceil
import matplotlib.patches as patches
import matplotlib.pylab as plt
import pandas as pd # for DatetimeIndex
import opensarlab_lib as asfn
from IPython.display import Markdown
from IPython.display import display

In [ ]:
!git clone https://github.com/markf6/pycorr_iceflow.git

# 1. Introduction to the Workflow We Will Use

## 1.1 The Basic Template Matching Approach Used Here

```pycorr_iceflow_1.1.py``` is based on the approach used in the production code of the <b><a href="https://nsidc.org/support/how/golive-map-application-user-guide" target="_blank">GoLIVE Landsat 8 processing</a></b> at NSIDC. It determines offsets at a user-specified grid spacing by comparing a square patch of pixels (a "chip") from an earlier image to the pixels in a larger square patch in a later image using the openCV ```cv2.matchTemplate``` function, a dft-based <b>cross correlation method</b> (see figure below) returning a correlation surface at integer pixel offsets between the two image chips. Sub-pixel offsets are determined by finding the peak of the spline of the correlation surface in the vicinity of the highest integer correlation peak.

## 1.2 Benefits of the PyCorr Algorithm

<img style="padding: 0px 0px 0px 10px" src="Figs/TemplateMatching.JPG" width="500" align="right" />```pycorr``` is a "relatively" light weight python script that exploits <b><a href="https://gdal.org/" target="_blank">GDAL</a></b> and <b><a href="https://opencv.org/" target="_blank">openCV</a></b> to rapidly determine offsets in an image pair. Because it uses GDAL for image I/O, it can use image pairs in many geospatial formats, with the caveat that the images do overlap some spatially and that their image pixels have the same size. 

pycorr produces a netCDF4 file with offsets and correlation values at a user-specified grid resolution in the same projection as the original images if the input images are in a UTM or Antarctic Polar Stereo (epsg:3031) projection - this is the set of projections used for Landsat imagery. If your images are in a a different projection, you are not out of luck - use the -```output_geotiffs_instead_of_netCDF``` option to output in the same projection as the input images - this option allows any projection GDAL knows about, which is most. The issue here is that the netCDF4 cf geolocation spec requires a variable block in the output file that is named after the projection, making it difficult to support all projections in a simple way.

There are a number of packages that provide similar analyses and may have more sophisticated approaches to identifying and filtering "noisy" matches, which can be due to cloud cover, surface change, low contrast features or an absence of features, shadows, and low signal-to-noise input imagery. ```pycorr``` is intentionally simple - it does not use a series of larger chip sizes if the initial match fails to find a peak at a location; it returns a limited set of metrics that help document the uniqueness and strength of a peak that can be used to filter the output, but it does not attempt to provide an error estimate for each match.

```pycorr``` is computationally fast because of the use of numpy and the openCV library, and so can process an image pair in minutes or tens of minutes depending on the image sizes and requested grid spacing and maximum offset specified for the search. This process can be sped up by using land and ocean masks to limit search distances off of the ice and also by using reference ice flow speed maps to set search distances larger in fast flowing areas and smaller in slow flowing areas, but for simplest uses these are not applied.

# 2. Exercising PyCorr on a Sentinel-2 Image Pair over Malaspina Glacier

## 2.1 Accessing the Sentinel-2 Data

We will use a pair of Sentinel-2 images acquired in May 2020 to demonstrate how ```pycorr``` works and which parameter it needs for creating an offset field. You can replicate this run by replacing the image pairs used below.

We first get two Sentinel 2 images from the <b><a href="https://registry.opendata.aws/sentinel-2-l2a-cogs/" target="_blank">AWS S2 Level2A CloudOptimizedGeotiff (COG) public data archive</a></b>

In [ ]:
!curl https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/7/V/EG/2020/5/S2B_7VEG_20200512_0_L2A/B08.tif --output S2B_7VEG_20200512_0_L2A_B08.tif
!curl https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/7/V/EG/2020/5/S2A_7VEG_20200527_0_L2A/B08.tif --output S2A_7VEG_20200527_0_L2A_B08.tif

<img style="padding: 0px 0px 0px 10px" src="Figs/Malaspina-Hubbard.jpg" width="500" align="right" />This image pair we will be analyzing in this part of the notebook covers a part of south-east Alaska including Malaspina and Hubbard glacier. 

<b>Malaspina glacier</b> is the largest piedmont glacier in the world. Situated at the head of the Alaska Panhandle, it is about 65 km (40 mi) wide and 45 km (28 mi) long, with an area of some 3,900 $km^2$ (1,500 $mi^2$). It is named in honor of Alessandro Malaspina, a Tuscan explorer in the service of the Spanish Navy, who visited the region in 1791. Malaspina has been undergoing a surge that started in around May 2020 and seems to have ended near the end of 2022. 

<b>Hubbard Glacier</b> is a glacier located in <b><a href="https://en.wikipedia.org/wiki/Wrangell%E2%80%93St._Elias_National_Park_and_Preserve" target="_blank">Wrangell–St. Elias National Park and Preserve</a></b> in eastern Alaska and Kluane National Park and Reserve in Yukon, Canada, and named after Gardiner Hubbard. The longest source for Hubbard Glacier originates 122 kilometers (76 mi) from its snout and is located at about 61°00′N 140°09′W, approximately 8 kilometers (5 mi) west of Mount Walsh with an elevation around 11,000 feet (3,400 m). A shorter tributary glacier begins at the easternmost summit on the Mount Logan ridge at about 18,300 feet (5,600 m) at about 60°35′0″N 140°22′40″W.

Before it reaches the sea, Hubbard is joined by the Valerie Glacier to the west, which, through forward surges of its own ice, has contributed to the advance of the ice flow that experts believe will eventually dam the Russell Fjord from Disenchantment Bay waters.

## 2.2 Running ```PyCorr``` to Perform Template Matching-based Glacier Velocity Mapping

We now will <b>run the ```pycorr``` algorithm</b> to perform template matching on the Sentinel-2 image pair. You'll see some warnings pop up early one. Don't mind those. 

In [ ]:
!python ./pycorr_iceflow/pycorr_iceflow_v1.1.py -imgdir . S2B_7VEG_20200512_0_L2A_B08.tif S2A_7VEG_20200527_0_L2A_B08.tif \
                              -img1datestr 20200512 -img2datestr 20200527 -datestrfmt "%Y%m%d" \
                              -inc 10 -half_source_chip 10  -half_target_chip 55 -plotvmax 25 -log10 \
                              -out_name_base S2A_S2BA_7VEG_15_20200512_20200527 -progupdates -use_itslive_land_mask_from_web

The code worked through the image pair and calculated thousands of offset points. It saved the output as a netCDF4 file, which we will look at in the following.

## 2.3 Visualizing ```PyCorr``` Template Matching Result for Our Area of Interest

We can now visualize the ```pycorr``` result to evaluate the performance of the algorithm. 

First we <b>import the netCDF4 file</b>:

In [ ]:
ds = nc.Dataset('S2A_S2BA_7VEG_15_20200512_20200527_hp.nc')

The output netCDF4 file (S2A_S2BA_7VEG_15_20200512_20200527.nc) includes various layers. We will choose the <b>vv_masked</b> layer to get ice flow speed. Alternatively you can read the <b>vx_masked</b> and <b>vy_masked</b> layers to get the vector components of the flow speed in projection x and y. Ice velocities are stored in the unit of <b>meters/day</b>.

Read the <b>vv_masked</b> layer

In [ ]:
vv_masked = np.ma.masked_where(ds['vv_masked']==0, ds['vv_masked'])

We will also <b>read the UTM coordinate information</b> out of the netCDF4 file for plotting purposes:

In [ ]:
UTMx = ds.variables['x'][:]
UTMy = ds.variables['y'][:]
TM = ds.variables['transverse_mercator'][:]

Now we can <b>visualize the velocity information</b> in an interactive plot 

In [ ]:
%matplotlib widget
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

plt.figure(figsize=(10, 7))
plt.title('Glacier Tracking Results', fontsize=16)
dx = (UTMx[1]-UTMx[0])/2.
dy = (UTMy[1]-UTMy[0])/2.
extent = [(UTMx[0]-dx)/1000.0, (UTMx[-1]+dx)/1000.0, (UTMy[0]-dy)/1000.0, (UTMy[-1]+dy)/1000.0]
plt.imshow(vv_masked*365.0, extent=extent, cmap='jet', vmin = 0, vmax = 3000)
plt.ylabel('UTM North [km]', fontsize=14)
plt.xlabel('UTM East [km]', fontsize=14)
plt.grid()
cbar = plt.colorbar()
#legend
cbar.set_label('Velocity in m/yr', rotation=270, fontsize=14)

# 3. Stack of Velocity Maps over Malaspina Glacier Generated using AutoRIFT

We have seen in the previous example how effective template matching can be for the application of glacier velocity mapping. BUT, we've also seen that the process of template matching is time consuming. Therefore, <b>offloading the process of ice velocities mapping to an automatic service</b> is appealing, especially when one is interested in mapping glacier motion across a longer time span. 

The Alaska Satellite Facility has recently made the template-tracking based AutoRIFT processor available through its search interface Vertex. To order AutoRIFT data, follow the following general workflow:
<ul>
  <li>Go to Vertex at <b><a href="https://search.asf.alaska.edu/" target="_blank">https://search.asf.alaska.edu/</a></b></li>
  <li>Search for Sentinel-1 Single Look Complex (SLC) images over your area of interest</li>
  <li>Use the SBAS search to create image pairs for AutoRIFT processing.</li>
  <li>Click on the "custom processing icon (three nested squares) to submit your jobs</li>
</ul>

## 3.1 Retrieving AutoRIFT Stack over Malaspina Glacier

Using the approach described above, I have ordered 1-year worth of AutoRIFT velocity maps from <b><a href="https://search.asf.alaska.edu/" target="_blank">ASF</a></b>. In total, I have collected 70 glacier velocity estimates over this time period, using 12-day and 24-day Sentinel-1 SAR pairs.

I have prepared these data sets and deposited them for you in an AWS S3 storage bucket. In a first step, we will <b>download these data and unzip them</b> into a folder called ```lab_6_data```.

In [ ]:
# Creating target folder
path = f"{os.getcwd()}/lab_6_data"
if not os.path.exists(path):
    os.makedirs(path)
os.chdir(path)
print(f"Current working directory: {os.getcwd()}")

In [ ]:
# Transfering data from Amazon AWS S3
malaspina_path = 's3://asf-jupyter-data-west/MalaspinaAutoRIFT.zip'
!aws --region=us-west-2 --no-sign-request s3 cp s3://asf-jupyter-data-west/MalaspinaAutoRIFT.zip MalaspinaAutoRIFT.zip 

In [ ]:
# Unzipping data
mala_path = f"{path}/MalaspinaAutoRIFT.zip"
asfn.asf_unzip(str(path), str(mala_path))

## 3.2 Create VRT And Explore Data Dimensions

We first <b>define two python functions</b> to identify all of our ```AutoRIFT``` Tiff files and to extract the image dates for these files. Because I am lazy, I am using the date of the reference image here. A more sophisticated approach could be to use the mean date between reference and secondary image.

In [ ]:
def get_tiff_paths(paths):
    tiff_paths = !ls $paths | sort -t_ -k5,5
    return tiff_paths

def get_dates(pths):
    dates = []
    for p in pths:
        for name_chunk in p.split('/')[-1].split('_'):
            nums = list(range(48, 58))
            if len(name_chunk) == 15 and ord(name_chunk[0]) in nums: 
                date = name_chunk.split('T')[0]
                dates.append(date)
                break
    dates.sort()
    return dates

Now we <b>use these functions</b> to identify our GeoTIFFs and extract associated image dates.

In [ ]:
paths = "*.tif*"
tiff_paths = get_tiff_paths(paths)

dates=get_dates(tiff_paths)

Now we <b>create a virtual raster table (VRT)</b> holding the information about all our GeoTIFFs.

In [ ]:
raster_path = 'raster_stack.vrt'
!gdalbuildvrt -separate $raster_path $paths

We also <b>create a Pandas time index</b> fpr our data and print the image dates. You see we have 70 image pairs in our stack with reference image times between Jan 09, 2021 and March 29, 2022

In [ ]:
time_index = pd.DatetimeIndex(dates)

for jacqdate, acqdate in enumerate(time_index):
    print('{:4d} {}'.format(jacqdate, acqdate.date()),end=' ')
    if (jacqdate % 5 == 4): print()

Now we <b>read the data</b> and mask out values associated with bad matches.

In [ ]:
img = gdal.Open(raster_path)
rasterstack = img.ReadAsArray()
rasterstack_masked = np.ma.masked_where(rasterstack<0, rasterstack)

## 3.3 Visualize Mean Glacier Velocity and Enable Extracting of Pixel-by-Pixel Velocity Time Series Extraction

We will now visualize an average velocity image in a way that we can move our mouse over the image and visualize the line/sample image coordinates. This will help us create time-series information for the most interesting image locations.

In [ ]:
temporal_mean = np.nanmean(rasterstack_masked, axis=0)

To do so, we first **create some helper functions:** 

In [ ]:
import warnings

warnings.filterwarnings('ignore')

class pixelPicker:
    def __init__(self, image, width, height):
        self.x = None
        self.y = None
        self.fig = plt.figure(figsize=(width, height))
        self.ax = self.fig.add_subplot(111, visible=False)
        self.rect = patches.Rectangle(
            (0.0, 0.0), width, height, 
            fill=False, clip_on=False, visible=False
        )
       
        self.rect_patch = self.ax.add_patch(self.rect)
        self.cid = self.rect_patch.figure.canvas.mpl_connect('button_press_event', 
                                                             self)
        self.image = image
        self.plot = self.gray_plot(self.image, fig=self.fig, return_ax=True)
        self.plot.set_title('Select a Point of Interest')
        
        
    def gray_plot(self, image, vmin=None, vmax=None, fig=None, return_ax=False):
        '''
        Plots an image in grayscale.
        Parameters:
        - image: 2D array of raster values
        - vmin: Minimum value for colormap
        - vmax: Maximum value for colormap
        - return_ax: Option to return plot axis
        '''
        if vmin is None:
            vmin = np.nanpercentile(self.image, 10)
        if vmax is None:
            vmax = np.nanpercentile(self.image, 99)
        if fig is None:
           my_fig = plt.figure() 
        ax = fig.add_axes([0.1,0.1,0.8,0.8])
        pos = ax.imshow(image, cmap= 'Blues' , interpolation='nearest', vmin=vmin, vmax=vmax)
        cbar = fig.colorbar(pos, ax=ax)
        cbar.set_label('Velocity in m/yr', rotation=270)
        if return_ax:
            return(ax)
        
    
    def __call__(self, event):
        print('click', event)
        self.x = event.xdata
        self.y = event.ydata
        for pnt in self.plot.get_lines():
            pnt.remove()
        plt.plot(self.x, self.y, 'ro')


Now we are ready to plot the average glacier velocity image. 

<div class="alert alert-success">
<font size="4"> <b>Click into the figure to select a point interest for which you want to analyze the glacier velocity time series.</b></font> 
<br> 
<font size="3"><b>Note</b>: This data set is in polar-stereo projection, giving it the rotated appearance.</font>
</div>

In [ ]:
fig_xsize = 7.5
fig_ysize = 7.5
my_plot = pixelPicker(temporal_mean, fig_xsize, fig_ysize)

**Save the selected coordinates:**

In [ ]:
sarloc = (ceil(my_plot.x), ceil(my_plot.y))
print(sarloc)

## 3.4 Plot Glacier Velocity Time Series at Point Locations

You've picked a location of interest above. Now, let's **pick a ```[3x3]```-sized rectangle around a center pixel which we selected ...**

In [ ]:
extent = (3, 3) # choose a 3 by 3 rectangle
bands = img.RasterCount

**... and extract the time series for this small area around the selected center pixel in a memory-efficient way (needed for larger stacks):**

In [ ]:
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
bs_aggregated = []
for band in range(bands):
    rs = img.GetRasterBand(band+1).ReadAsArray(sarloc[0], sarloc[1], 
                                               extent[0], extent[1])
    rs_masked = np.ma.masked_where(rs<0, rs)
    rs_mean = np.nanmean(rs_masked)
    bs_aggregated.append(rs_mean)

fig, ax = plt.subplots(1, 1, figsize=(10, 5))
plt.title('Glacier Velocity Results', fontsize=16)
ax.scatter(time_index, bs_aggregated, color='k', marker='o')
ax.set_xlabel('Date', fontsize=14)
ax.set_ylabel('Glacier Velocity [m/day]', fontsize=14)
plt.xticks(rotation = 45)
ax.set_ylim([0, 4500])
plt.grid()
fig.tight_layout() 

<div class="alert alert-success">
<font size="4"> <b>Please Explore Several Points on the Map by going back to the average velocity figure, selecting a different point, and running the code cells under the average velocity figure one more time.</b></font> 
<br> 
</div>

# 4 Version Log

<font face="Calibri" size="2"> <i>GEOS 639 Geodetic Imaging - Version 1.0.0 -  April 2022 
    <br>
        <b>Version Changes:</b>
    <ul>
        <li>First version of this lab</li>
    </ul>
    </i>
</font>